# Steps of msm_we:

![Steps of msm_we](MSM_steps.png)

# Compatibility of msm_we: NESS Simulation Data

![NESS_Equ_Prob](NESS_Equl_prob.png)

#### If running with $OMP_NUM_THREADS > 1, Ray parallelism may occasionally silently hang during clustering/flux matrix calculations. If submitting the msm_we on a cluster, then set the variable export OMP_NUM_THREADS=1 in the SLURM file used to submit the job.

In [1]:
%env OMP_NUM_THREADS=1

env: OMP_NUM_THREADS=1


In [2]:
import msm_we
import numpy as np
import pickle
import ray

#### Here, Ray is initialized and the number of CPUs employed for the msm_we job is specified.

In [3]:
ray.init(num_cpus=2)  #Example if 2 cpus are used

2024-06-18 08:36:48,071	INFO worker.py:1749 -- Started a local Ray instance.


Python version:,3.10.14
Ray version:,2.23.0


## Prep
#### First, let’s set some parameters for haMSM building.

In [4]:
#ModelName
model_name = 'NTL9_SynMD_WEfolding'

#west.h5 file location
h5file_paths = ['../ntl9_sample_files/completed_files/west.h5']

# Reference structure
ref_file = '../ntl9_sample_files/ntl9_folding_synd/ntl9.pdb'

#### Basis and Target need to be defined as done while running WE-NESS simulations
##### Currently, the msm_we package supports the Basis and Target definition along the first progress coordinate.
##### All structures in Basis (Target) WE bins are grouped in one cluster when building the transition matrix.

In [5]:
# Boundaries of the basis/target, in progress coordinate space
pcoord_bounds = { 'basis':[[10.0, 20.0]], 'target':[[0, 1.0]] }

#### Tau needs to be the resampling time (in second) used in the WE simulation.

In [6]:
# WESTPA resampling time (in second):
tau = 3e-10 

#### Define MSM Featurization
##### The function 'processCoordinates' defines a transformation that’s applied to the coordinates in 'auxdata/coord' before dimensionality reduction.
##### The inputs are the coordinates (read from 'auxdata/coord') for each segment in the loaded iteration(s), and it should return an array of features for each.
##### For example: Calpha pairwise distnaces.

In [7]:
import Coordinate_Processing
msm_we.modelWE.processCoordinates = Coordinate_Processing.processCoordinates

## Model Initialization parameters
#### modelName: str,
#### fileSpecifier: str,
#### refPDBfile: str,
#### basis_pcoord_bounds: list = None,
#### target_pcoord_bounds: list = None,
#### dim_reduce_method: str = "none",  
##### 'pca', 'tica', 'vamp'
#### tau: float = None,
#### pcoord_ndim: int = 1, If more than 1 pcoord_ndim is there, msm_we will take first one to.....
#### auxpath: str = "coord",
#### use_weights_in_clustering=False,

In [8]:
#DimReduction method
dimreduce_method = 'vamp'

In [9]:
model = msm_we.modelWE()
model.initialize(modelName=model_name,fileSpecifier=h5file_paths,refPDBfile=ref_file,basis_pcoord_bounds=pcoord_bounds['basis'],target_pcoord_bounds=pcoord_bounds['target'],tau=tau,dim_reduce_method=dimreduce_method)

In [10]:
model.get_iterations()    #Get total WE iteration
model.get_coordSet(last_iter = model.maxIter)    #loads data 

Getting coordSet ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 0:00:07

## Dimensionality Reduction Call Options
#### first_iter=1: Training data starts from the first WE iteration.
#### last_iter=None: Specifies up to which iteration. If not specified, defaults to the last WE iteration.
#### fine_stride=1, interval...
#### variance_cutoff=0.95 (default value), but can be specified.
#### use_weights=True (default value), but can be specified as False.

In [11]:
model.dimReduce(variance_cutoff=0.05, use_weights=False)
with open('Outputs/Dim_Reduce_Vamp_05.obj', 'wb') as outfile:
    pickle.dump(model, outfile)

Loading data for VAMP ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 0:00:08

[06/18/24 08:37:25] INFO     Performing weighted vamp                                        ]8;id=889586;file:///home/groups/ZuckermanLab/kanias/Other_Downloads/westpa-workshop2024-tutorials-main/tutorial3-rates/msm_we-main/msm_we/_hamsm/_dimensionality.py\_dimensionality.py]8;;\:]8;id=547128;file:///home/groups/ZuckermanLab/kanias/Other_Downloads/westpa-workshop2024-tutorials-main/tutorial3-rates/msm_we-main/msm_we/_hamsm/_dimensionality.py#300\300]8;;\

[06/18/24 08:37:26] INFO      vamp will reduce 741 to 13 components.                         ]8;id=757302;file:///home/groups/ZuckermanLab/kanias/Other_Downloads/westpa-workshop2024-tutorials-main/tutorial3-rates/msm_we-main/msm_we/_hamsm/_dimensionality.py\_dimensionality.py]8;;\:]8;id=748560;file:///home/groups/ZuckermanLab/kanias/Other_Downloads/westpa-workshop2024-tutorials-main/tutorial3-rates/msm_we-main/msm_we/_hamsm/_dimensionality.py#316\316]8;;\

## Clustering call options
#### n_clusters, number of clusters per stratum
#### use_ray=False,
#### stratified=True, If False, it performs aggregated clustering (not recommended).
#### iters_to_use=None: If None, then all WE iteration data is used for training Example: iters_to_use=range(1,10) 
#### store_validation_model=False: If True, then it can perform block validation (explained later).
#### **_cluster_args, Arguments for the clustering algorithm (sklearn.cluster.MiniBatchKMeans).

In [12]:
# Number of MSM microstates to initially put in each stratum/WE bin
clusters_per_stratum = 3

model.cluster_coordinates(n_clusters=clusters_per_stratum, use_ray=True, streaming=True, stratified=True, store_validation_model=True, random_state=1337, max_iter=200)

with open('Outputs/StratifiedClustering_3perStarta.obj', 'wb') as outfile:
    pickle.dump(model, outfile)

Clustering ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 0:00:08

## get_fluxMatrix call options: To build flux matrix
#### n_lag=0: Only this is supported, meaning building the flux matrix for the lag time as WE resampling time (tau).
#### iters_to_use=None: If None, then all WE iteration data is used for training. Example: iters_to_use=range(1,10).

In [13]:
model.get_fluxMatrix(n_lag=0)
with open('Outputs/FluxMatrixRaw.obj', 'wb') as outfile:
    pickle.dump(model, outfile)

Constructing flux matrix ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 0:00:27

## organize_fluxMatrix: Remove disconnected clusters

In [14]:
model.organize_fluxMatrix()
with open('Outputs/FluxMatrixOrganized.obj', 'wb') as outfile:
    pickle.dump(model, outfile)

Constructing flux matrix ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 0:00:00 0:00:27

[06/18/24 08:39:10] INFO     Note: Sorting bins, assuming that pcoord 0 is meaningful for       ]8;id=99660;file:///home/groups/ZuckermanLab/kanias/Other_Downloads/westpa-workshop2024-tutorials-main/tutorial3-rates/msm_we-main/msm_we/_hamsm/_clustering.py\_clustering.py]8;;\:]8;id=936913;file:///home/groups/ZuckermanLab/kanias/Other_Downloads/westpa-workshop2024-tutorials-main/tutorial3-rates/msm_we-main/msm_we/_hamsm/_clustering.py#1567\1567]8;;\
                             sorting                                                                               

## Further calls to:
#### Build the transition matrix
#### Estimate steady-state probability
#### Estimate steady-state target flux

In [15]:
model.get_Tmatrix()
model.get_steady_state()
model.get_steady_state_target_flux()

[06/18/24 08:39:15] INFO                                                                           ]8;id=598465;file:///home/groups/ZuckermanLab/kanias/Other_Downloads/westpa-workshop2024-tutorials-main/tutorial3-rates/msm_we-main/msm_we/_hamsm/_analysis.py\_analysis.py]8;;\:]8;id=756926;file:///home/groups/ZuckermanLab/kanias/Other_Downloads/westpa-workshop2024-tutorials-main/tutorial3-rates/msm_we-main/msm_we/_hamsm/_analysis.py#179\179]8;;\
                             Flux converged to 6.1183e+05 after 1 iterations of inverse iteration.                 

In [16]:
with open('Outputs/SteadyState_Flux.obj', 'wb') as outfile:
    pickle.dump(model, outfile)